In [208]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [209]:
url = "https://en.wikipedia.org/wiki/2025_Indian_Premier_League"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")
data_records = []
print(soup.title.text)

200
2025 Indian Premier League - Wikipedia


In [210]:
matches = soup.find_all("div", style="width: 100%; clear:both")

In [214]:
#Scrapper
table2data = []
for match in matches:
    tables = match.find_all("table")
    table1_data = tables[0].get_text("\n", strip=True).split("\n")
    match_no = table1_data[0]
    match_date = table1_data[1]
    match_date = match_date.replace("\xa0", " ")
    match_time = None
    
    for item in table1_data:
        if ":" in item:
            match_time = item.replace("(" , "").strip()
            break

    table2_data = tables[1].get_text("\n", strip=True).split("\n")
        
    if (len(table2_data) <= 4):
        team1 = table2_data[0]
        score1 = "0"
        team2 = table2_data[3]
        score2 = "0"
    else:
        team1 = table2_data[0]
        score1 = table2_data[2]
        team2 = table2_data[4]
        score2 = table2_data[5]
    table2data.append(table2_data)
    table3_data = tables[2].get_text("\n", strip=True).split("\n")
    venue = table3_data[1]
    result = table3_data[0]
    winner = result.split(" won")[0]
    if len(table3_data) == 7:
        pom = "no result"
    else:
        pom = table3_data[len(table3_data) - 2]
    
    toss_winner = "N/A"
    toss_desicion = "N/A"
    for li in match.select("li"):
        text = li.get_text(strip=True)
        if "won the toss" in text:
            toss_winner = text.split(" won the toss")[0]
            toss_decision = text.split(" to ")[1]
            break
            
    #Data Records
    data_records.append({
        "match_no":match_no,
        "date":match_date,
        "time":match_time,
        "venue":venue,
        "toss_winner": toss_winner,
        "toss_decision":toss_decision,
        "team1":team1,
        "team2":team2,
        "team1_score":score1,
        "team2_score":score2,
        "winner":winner,
        "result":result,
        "POM":pom
    })


with open("data.txt", "w", encoding="utf-8") as f:
    for item in table2data:
        f.write(item + "\n")

TypeError: can only concatenate list (not "str") to list